## Подготовка данных

### Импорт библиотек

In [1]:
import os
import sys
from pathlib import Path

# Добавляем путь на уровень выше
sys.path.append(str(Path(os.getcwd()).resolve().parent))

from utils.features import *
from utils.load_data import load_all_data
from utils.graph_features import GraphClusterProcessor
from models.dfdgcn import DFDGCN

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import networkx as nx
from node2vec import Node2Vec

import time
from tqdm import tqdm
from sklearn.cluster import KMeans

### Загрузка данных

In [2]:
data_dir = Path('../data/PEMS-BAY')
metadata, data, adj = load_all_data(data_dir)
data = data[:2016]

### GraphClusterProcessor


In [3]:
processor = GraphClusterProcessor(adj, data)
processor.elbow_method(metric='closeness', max_clusters=15)

labels = KMeans(n_clusters=10, random_state=42).fit_predict(
    np.array(list(nx.betweenness_centrality(nx.from_numpy_array(adj)).values())).reshape(-1, 1)
)
processor.plot_group_average_speeds(labels, n_clusters=10)

In [4]:
data_expanded = processor.cluster_and_add_channel(metric='closeness', n_clusters=10, one_hot=True)

### Node Emb

In [105]:
node_indices = np.array([i for i in range(data.shape[1])])
node_indices_expanded = np.expand_dims(node_indices, axis=0)  # Форма: (1, 325,)
node_indices_expanded = np.expand_dims(node_indices_expanded, axis=-1)  # Форма: (1, 325, 1)
node_indices_expanded = np.repeat(node_indices_expanded, data.shape[0], axis=0)  # Форма: (2016, 325, 1)

# Объединение с временными рядами по последней оси
data_expanded = np.concatenate([data, node_indices_expanded], axis=-1)  # Форма: (2016, 325, 3 + 64)

# Проверка результата
print("Исходные данные (временные ряды):", data.shape)
print("Эмбеддинги узлов:", node_indices.shape)
print("Объединённые данные:", data_expanded.shape)

Исходные данные (временные ряды): (2016, 325, 3)
Эмбеддинги узлов: (325,)
Объединённые данные: (2016, 325, 4)


### Node2vec embeddings

In [ ]:
np.fill_diagonal(adj, 0)
G = nx.from_numpy_array(adj)
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=400, p=1, q=1, workers=12)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
node_embeddings = np.array([model.wv[str(node)] for node in G.nodes()])

In [ ]:
node_embeddings_expanded = np.expand_dims(node_embeddings, axis=0)  # Форма: (1, 325, 64)
node_embeddings_expanded = np.repeat(node_embeddings_expanded, data.shape[0], axis=0)  # Форма: (2016, 325, 64)

# 4. Объединение с временными рядами по последней оси
data_expanded = np.concatenate([data, node_embeddings_expanded], axis=-1)  # Форма: (2016, 325, 3 + 64)

# 5. Проверка результата
print("Исходные данные (временные ряды):", data.shape)
print("Эмбеддинги узлов:", node_embeddings.shape)
print("Объединённые данные:", data_expanded.shape)

### Position Encoding (PE)

In [ ]:
def positional_encoding(pos, d_model):
    position = np.arange(pos)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    pe = np.zeros((pos, d_model))
    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)
    return pe

# Пример для 325 узлов
num_nodes = data.shape[1]
d_model = 16
pe = positional_encoding(num_nodes, d_model)  # Форма: (325, 64)

# Расширение до (batch_size, 325, 64)
pe = positional_encoding(num_nodes, d_model).astype(np.float32)  # Используйте float32
pe_expanded = np.expand_dims(pe, axis=0)  # Форма: (1, 325, 64)
pe_expanded = np.repeat(pe_expanded, data.shape[0], axis=0)  # Форма: (2016, 325, 64)
pe_expanded = torch.tensor(pe_expanded, dtype=torch.float32)     # Для тензоров

data_expanded = np.concatenate([data, pe_expanded], axis=-1)  # Форма: (2016, 325, 3 + 64)

# 5. Проверка результата
print("Исходные данные (временные ряды):", data.shape)
print("Эмбеддинги узлов:", pe_expanded.shape)
print("Объединённые данные:", data_expanded.shape)

### Разделение данных и создание Dataloader

In [5]:
# Данные и параметры
L, N, C = data_expanded.shape  # [2016, 325, C]
batch_size = 16
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2
seq_len = 12  # Количество временных шагов на вход
pred_len = 12  # Количество временных шагов для предсказания

# Индексы каналов для нормализации
normalize = True

channels_to_normalize = [0]

# Проверка корректности разделения данных
assert train_ratio + val_ratio + test_ratio == 1.0, "Сумма долей train, val и test должна быть равна 1.0"

# Разделение данных на train, val и test
num_samples = data_expanded.shape[0]  # Количество временных шагов (L)
train_size = int(num_samples * train_ratio)
val_size = int(num_samples * val_ratio)
test_size = num_samples - train_size - val_size

train_data = data_expanded[:train_size, :, :]  # [train_size, N, C]
val_data = data_expanded[train_size:train_size + val_size, :, :]  # [val_size, N, C]
test_data = data_expanded[train_size + val_size:, :, :]  # [test_size, N, C]

# Убедитесь, что данные имеют разрешение на запись
train_data = train_data.copy()
val_data = val_data.copy()
test_data = test_data.copy()

# Нормализация данных
if normalize:
    assert all(0 <= ch < C for ch in channels_to_normalize), "Индексы каналов выходят за пределы допустимого диапазона"
    channel_max = train_data[:, :, channels_to_normalize].max(axis=(0, 1), keepdims=True)  # Форма [1, 1, len(channels_to_normalize)]
    channel_max[channel_max == 0] = 1.0
    train_data[:, :, channels_to_normalize] = train_data[:, :, channels_to_normalize] / channel_max
    val_data[:, :, channels_to_normalize] = val_data[:, :, channels_to_normalize] / channel_max
    test_data[:, :, channels_to_normalize] = test_data[:, :, channels_to_normalize] / channel_max

# Создание кастомного Dataset
class TrafficDataset(Dataset):
    def __init__(self, data, seq_len, pred_len):
        super().__init__()
        self.data = data  # Форма [L, N, C]
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        # Количество возможных последовательностей
        return self.data.shape[0] - self.seq_len - self.pred_len + 1

    def __getitem__(self, idx):
        # Извлекаем последовательность входных данных
        x = self.data[idx:idx + self.seq_len, :, :]  # Форма [seq_len, N, C]
        # Извлекаем целевую последовательность
        y = self.data[idx + self.seq_len:idx + self.seq_len + self.pred_len, :, 0]  # Форма [pred_len, N, C]
        return x, y
    
# Преобразование данных в тензоры с dtype=torch.float32
train_data = torch.tensor(train_data, dtype=torch.float32)
val_data = torch.tensor(val_data, dtype=torch.float32)
test_data = torch.tensor(test_data, dtype=torch.float32)

# Создание DataLoader
train_dataset = TrafficDataset(train_data, seq_len, pred_len)
val_dataset = TrafficDataset(val_data, seq_len, pred_len)
test_dataset = TrafficDataset(test_data, seq_len, pred_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
for x, y in train_loader:
    print(x.shape, y.shape)
    print(x[0, 0, :2, :])
    print(y[:2, :2, :2])
    print(x[0, 0, 0, 0].dtype)
    print(x[0, 0, 0, 1].dtype)
    print(x[0, 0, 0, 2].dtype)
    print(x[0, 0, 0, 3].dtype)
    1/0

torch.Size([16, 12, 325, 13]) torch.Size([16, 12, 325])
tensor([[0.8665, 0.6840, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.4709, 0.6840, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000]])
tensor([[[0.8556, 0.4600],
         [0.8495, 0.4527]],

        [[0.8374, 0.2646],
         [0.8240, 0.2731]]])
torch.float32
torch.float32
torch.float32
torch.float32


ZeroDivisionError: division by zero

## Инициализация моделей и запуск обучения

### QGNN Model

In [8]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

def normalize_directed_adj(adj, mode='row'):
    """
    Нормализация для ориентированного графа.
    
    Args:
        adj (torch.Tensor): Матрица смежности (N x N).
        mode (str): 'row' для нормализации по строкам, 'col' для нормализации по столбцам.
    
    Returns:
        torch.Tensor: Нормализованная матрица смежности.
    """
    adj = adj + torch.eye(adj.size(0)).to(adj.device)
    if mode == 'row':
        degree = torch.sum(adj, dim=1, keepdim=True)
    elif mode == 'col':
        degree = torch.sum(adj, dim=0, keepdim=True)
    else:
        raise ValueError("Mode must be 'row' or 'col'.")
    
    degree_inv = torch.where(degree > 0, 1.0 / degree, torch.zeros_like(degree))
    return adj * degree_inv

def normalize_adj(adj):
    """
    Нормализует матрицу смежности симметричным способом.
    
    Args:
        adj (torch.Tensor): Матрица смежности (N x N).
    
    Returns:
        torch.Tensor: Нормализованная матрица смежности.
    """
    # Добавляем петли (self-loops) к матрице смежности
    adj = adj + torch.eye(adj.size(0)).to(adj.device)
    
    # Вычисляем степени узлов (degree matrix)
    degree = torch.sum(adj, dim=1)
    
    # Симметричная нормализация: D^(-1/2) * A * D^(-1/2)
    degree_inv_sqrt = torch.diag(torch.pow(degree, -0.5))
    adj_normalized = torch.mm(torch.mm(degree_inv_sqrt, adj), degree_inv_sqrt)
    
    return adj_normalized

# Функция для создания матрицы Гамильтона
def make_quaternion_mul(kernel):
    dim = kernel.size(1) // 4
    r, i, j, k = torch.split(kernel, [dim, dim, dim, dim], dim=1)
    hamilton = torch.cat([
        torch.cat([r, -i, -j, -k], dim=0),
        torch.cat([i, r, -k, j], dim=0),
        torch.cat([j, k, r, -i], dim=0),
        torch.cat([k, -j, i, r], dim=0)
    ], dim=1)
    return hamilton

# Слой QGNN
class QGNNLayer(nn.Module):
    def __init__(self, in_features, out_features, num_nodes, space, dropout=0.5, act=F.tanh):
        super(QGNNLayer, self).__init__()
        self.num_nodes = num_nodes
        self.space = space
        self.act = act
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(out_features)

        # Проверяем кратность in_features
        if self.space == 'q':
            assert in_features % 4 == 0, "in_features must be divisible by 4 for quaternion operations"
            self.weight = Parameter(torch.FloatTensor(in_features // 4, out_features))
        else:
            self.weight = Parameter(torch.FloatTensor(in_features, out_features))

        self.dim = self.weight.size(1) // 4
        self.reset_parameters()

    def reset_parameters(self):
        stdv = math.sqrt(6.0 / (self.weight.size(0) + self.weight.size(1)))
        self.weight.data.uniform_(-stdv, stdv)

    def forward(self, x, adj):
        x = self.dropout(x)
        if self.space == 'q':
            hamilton = make_quaternion_mul(self.weight)
            support = torch.mm(x, hamilton)  # Hamilton product
        else:
            support = torch.mm(x, self.weight)
        
        B_N, hidden_dim = support.shape
        B = B_N // self.num_nodes
        N = self.num_nodes
        support = support.view(B, N, hidden_dim)

        # Обработка каждого батча с использованием torch.bmm
        output = torch.bmm(adj.unsqueeze(0).expand(B, -1, -1), support)
        output = output.view(B * N, hidden_dim)
        output = self.bn(output)
        output = self.act(output)

        return output


# Основная модель для прогнозирования трафика
class QGNNTrafficPredictor(nn.Module):
    def __init__(self, adj, 
                 num_nodes, 
                 input_dim, 
                 hidden_dim, 
                 output_dim, 
                 num_layers, 
                 pre_len, 
                 space, 
                 emb_dim,
                 dropout=0.5, 
                 directed=False,
                 add_c=0,):
        super(QGNNTrafficPredictor, self).__init__()
        self.adj = normalize_directed_adj(adj) if directed else normalize_adj(adj)
        self.pre_len = pre_len
        self.num_nodes = num_nodes
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.dropout = dropout

        # Убедимся, что input_dim кратно 4
        if space == 'q':
            assert input_dim % 4 == 0, "input_dim must be divisible by 4 for quaternion operations"

        # Инициализация эмбеддингов
        self.embedding = nn.Embedding(emb_dim, 5)    # Эмбеддинг для D_i_W (7 вариантов)
        self.T_i_D_emb = nn.Embedding(288, 5)        # Эмбеддинг для T_i_D (288 вариантов)
        self.D_i_W_emb = nn.Embedding(7, 5)          # Эмбеддинг для D_i_W (7 вариантов)

        # QGNN (GCN) слои
        self.qgnn_layers = nn.ModuleList([
            QGNNLayer(input_dim if i == 0 else hidden_dim, hidden_dim, self.num_nodes, space, dropout)
            for i in range(num_layers)
        ])

        # Временной слой (GRU)
        self.temporal_layer = nn.GRU(hidden_dim + add_c, hidden_dim, batch_first=True)
        self.relu = F.relu
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, history_data):
        B, L, N, C = history_data.shape
        history_data = history_data.permute(0, 2, 1, 3)  # [B, N, L, C]
        speed = history_data[:, :, :, :1]  # [B, N, L, input_dim]
        # embedding_indices = torch.round(history_data[:, :, :, -1]).type(torch.LongTensor).to(history_data.device)

        # Применение эмбеддингов
        # embedding = self.embedding(embedding_indices)  

        outputs = []
        for t in range(L):
            x = history_data[:, :, t, :1]  # [B, N, C]
            x = x.reshape(B * N, -1)  # [B * N, C]

            for i, qgnn_layer in enumerate(self.qgnn_layers):
                x = qgnn_layer(x, self.adj)  # [B * N, hidden_dim]

            x = x.reshape(B, N, -1)  # [B, N, hidden_dim]
            outputs.append(x)

        # Объединяем выходы по временным шагам
        outputs = torch.stack(outputs, dim=2)  # [B, N, L, hidden_dim]
        outputs = torch.cat([speed,outputs,history_data[:, :, :, -10:]], dim=-1)  # [B, N, L, hidden_dim + channels]

        # Применяем временной слой (GRU)
        outputs = outputs.reshape(B * N, L, -1)  # [B * N, L, hidden_dim + channels]
        outputs, _ = self.temporal_layer(outputs)  # [B * N, L, hidden_dim]
        outputs = outputs.reshape(B, N, L, -1)  # [B, N, L, hidden_dim]
        outputs = outputs[:, :, -self.pre_len:, :]  # [B, N, pre_len, hidden_dim]
        outputs = self.relu(outputs)

        # Применяем выходной слой
        outputs = self.fc(outputs)  # [B, N, pre_len, output_dim]
        return outputs.permute(0, 2, 1, 3)  # [B, pre_len, N, output_dim]

### DFDGCN Model

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Scaled dot-product attention
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

class convt(nn.Module):
    def __init__(self):
        super(convt, self).__init__()

    def forward(self, x, w):
        x = torch.einsum('bne, ek->bnk', (x, w))
        return x.contiguous()
    
class nconv(nn.Module):
    def __init__(self):
        super(nconv, self).__init__()

    def forward(self, x, A, dims):
        if dims == 2:
            x = torch.einsum('ncvl,vw->ncwl', (x, A))
        elif dims == 3:
            x = torch.einsum('ncvl,nvw->ncwl', (x, A))
        else:
            raise NotImplementedError('DFDGCN not implemented for A of dimension ' + str(dims))
        return x.contiguous()

class linear(nn.Module):
    """Linear layer."""

    def __init__(self, c_in, c_out):
        super(linear, self).__init__()
        self.mlp = torch.nn.Conv2d(c_in, c_out, kernel_size=(
            1, 1), padding=(0, 0), stride=(1, 1), bias=True)

    def forward(self, x):
        return self.mlp(x)

class gcn(nn.Module):
    """Graph convolution network."""

    def __init__(self, c_in, c_out, dropout, support_len=3, order=2):
        super(gcn, self).__init__()
        self.nconv = nconv()

        self.c_in = c_in
        c_in = (order * (support_len + 1) + 1) * self.c_in
        self.mlp = linear(c_in, c_out)
        self.dropout = dropout
        self.order = order

    def forward(self, x, support):

        out = [x]
        for a in support:
            x1 = self.nconv(x, a.to(x.device), a.dim())
            out.append(x1)

            for k in range(2, self.order + 1):
                x2 = self.nconv(x1, a.to(x1.device), a.dim())
                out.append(x2)
                x1 = x2
        h = torch.cat(out, dim=1)
        h = self.mlp(h)
        h = F.dropout(h, self.dropout, training=self.training)
        return h

def dy_mask_graph(adj, k):
    M = []
    for i in range(adj.size(0)):
        adp = adj[i]
        mask = torch.zeros( adj.size(1),adj.size(2)).to(adj.device)
        mask = mask.fill_(float("0"))
        s1, t1 = (adp + torch.rand_like(adp) * 0.01).topk(k, 1)
        mask = mask.scatter_(1, t1, s1.fill_(1))
        M.append(mask)
    mask = torch.stack(M,dim=0)
    adj = adj * mask
    return adj

def cat(x1,x2):
    M = []
    for i in range(x1.size(0)):
        x = x1[i]
        new_x = torch.cat([x,x2],dim=1)
        M.append(new_x)
    result = torch.stack(M,dim=0)
    return result

class DFDGCN(nn.Module):

    def __init__(self, num_nodes, dropout=0.3, supports=None,
                    gcn_bool=True, addaptadj=True, aptinit=None,
                    in_dim=2, out_dim=12, residual_channels=32,
                    dilation_channels=32, skip_channels=256, end_channels=512,
                    kernel_size=2, blocks=4, layers=2, a=1, seq_len=12, affine=True, 
                    fft_emb=10, identity_emb=10, hidden_emb=30, subgraph=20, 
                    add_channels=0, embed_size=64, heads=4):
        super(DFDGCN, self).__init__()
        self.a = a
        self.heads = heads
        self.emb = fft_emb
        self.blocks = blocks
        self.layers = layers
        self.seq_len = seq_len
        self.dropout = dropout
        self.supports = supports
        self.gcn_bool = gcn_bool
        self.addaptadj = addaptadj
        self.hidden_emb = hidden_emb
        self.embed_size = embed_size
        self.subgraph_size = subgraph
        self.add_channels = add_channels
        self.identity_emb = identity_emb
        self.bn = nn.ModuleList()
        self.gconv = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        self.skip_convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.residual_convs = nn.ModuleList()
        self.self_attention = SelfAttention(embed_size, heads)
        self.start_conv = nn.Conv2d(in_channels=in_dim,
                                    out_channels=residual_channels,
                                    kernel_size=(1, 1))

        self.fft_len = round(seq_len//2) + 1
        self.Ex1 = nn.Parameter(torch.randn(self.fft_len, self.emb), requires_grad=True)
        self.Wd = nn.Parameter(torch.randn(num_nodes,self.emb + self.identity_emb + self.seq_len * 2 + self.add_channels, self.hidden_emb), requires_grad=True)
        self.Wxabs = nn.Parameter(torch.randn(self.hidden_emb, self.hidden_emb), requires_grad=True)
        self.pe = nn.Parameter(torch.randn(embed_size, embed_size), requires_grad=True)

        self.mlp = linear(residual_channels * 4,residual_channels)
        self.layersnorm = torch.nn.LayerNorm(normalized_shape=[num_nodes,self.hidden_emb], eps=1e-08,elementwise_affine=affine)
        self.convt = convt()

        self.node1 = nn.Parameter(
            torch.randn(num_nodes, self.identity_emb), requires_grad=True)
        self.drop = nn.Dropout(p=dropout)

        self.T_i_D_emb = nn.Parameter(
            torch.empty(288, self.seq_len))
        self.D_i_W_emb = nn.Parameter(
            torch.empty(7, self.seq_len))
        self.G_emb = nn.Parameter(
            torch.empty(num_nodes, self.seq_len))

        receptive_field = 1
        self.reset_parameter()
        self.supports_len = 0
        if not addaptadj:
            self.supports_len -= 1
        if supports is not None:
            self.supports_len += len(supports)
        if gcn_bool and addaptadj:
            if aptinit is None:
                if supports is None:
                    self.supports = []
                self.nodevec1 = nn.Parameter(
                    torch.randn(num_nodes, self.emb), requires_grad=True)
                self.nodevec2 = nn.Parameter(
                    torch.randn(self.emb, num_nodes), requires_grad=True)
                self.supports_len += 1
            else:
                if supports is None:
                    self.supports = []
                m, p, n = torch.svd(aptinit)
                initemb1 = torch.mm(m[:, :10], torch.diag(p[:10] ** 0.5))
                initemb2 = torch.mm(torch.diag(p[:10] ** 0.5), n[:, :10].t())
                self.nodevec1 = nn.Parameter(initemb1, requires_grad=True)
                self.nodevec2 = nn.Parameter(initemb2, requires_grad=True)
                self.supports_len += 1

        for b in range(blocks):
            additional_scope = kernel_size - 1
            new_dilation = 1
            for i in range(layers):
                # dilated convolutions
                self.filter_convs.append(nn.Conv2d(in_channels=residual_channels,
                                                   out_channels=dilation_channels,
                                                   kernel_size=(1, kernel_size), dilation=new_dilation))

                self.gate_convs.append(nn.Conv2d(in_channels=residual_channels,
                                                 out_channels=dilation_channels,
                                                 kernel_size=(1, kernel_size), dilation=new_dilation))

                # 1x1 convolution for residual connection
                self.residual_convs.append(nn.Conv2d(in_channels=dilation_channels,
                                                     out_channels=residual_channels,
                                                     kernel_size=(1, 1)))

                # 1x1 convolution for skip connection
                self.skip_convs.append(nn.Conv2d(in_channels=dilation_channels,
                                                 out_channels=skip_channels,
                                                 kernel_size=(1, 1)))
                self.bn.append(nn.BatchNorm2d(residual_channels))
                new_dilation *= 2
                receptive_field += additional_scope
                additional_scope *= 2
                if self.gcn_bool:
                    self.gconv.append(
                        gcn(dilation_channels, residual_channels, dropout, support_len=self.supports_len))
        self.end_conv_1 = nn.Conv2d(in_channels=skip_channels,
                                    out_channels=end_channels,
                                    kernel_size=(1, 1),
                                    bias=True)

        self.end_conv_2 = nn.Conv2d(in_channels=end_channels,
                                    out_channels=out_dim,
                                    kernel_size=(1, 1),
                                    bias=True)

        self.receptive_field = receptive_field

    def reset_parameter(self):
        nn.init.xavier_uniform_(self.T_i_D_emb)
        nn.init.xavier_uniform_(self.D_i_W_emb)
        nn.init.xavier_uniform_(self.G_emb)


    def forward(self, history_data: torch.Tensor) -> torch.Tensor:
        """Feedforward function of DFDGCN; Based on Graph WaveNet

        Args:
            history_data (torch.Tensor): shape [B, L, N, C]

        Graphs:
            predefined graphs: two graphs; [2, N, N] : Pre-given graph structure, including in-degree and out-degree graphs

            self-adaptive graph: [N, N] : Self-Adaptively constructed graphs with two learnable parameters
                torch.mm(self.nodevec1, self.nodevec2)
                    nodevec: [N, Emb]

            dynamic frequency domain graph: [B, N, N] : Data-driven graphs constructed with frequency domain information from traffic data
                traffic_data : [B, N, L]
                frequency domain information : [B, N, L/2.round + 1] ------Embedding ------[B, N, Emb2]
                Identity embedding : learnable parameter [N, Emb3]
                Time embedding : Week and Day : [N, 7] [N, 24(hour) * 12 (60min / 5min due to sampling)] ------Embedding ------ [N, 2 * Emb4]
                Concat frequency domain information + Identity embedding + Time embedding ------Embedding , Activating, Normalization and Dropout
                Conv1d to get adjacency matrix

        Returns:
            torch.Tensor: [B, L, N, 1]
        """
        #num_feat = model_args["num_feat"]
        input = history_data.transpose(1, 3).contiguous()[:,:,:,:]
        
        data = history_data


        in_len = input.size(3)
        if in_len < self.receptive_field:
            x = nn.functional.pad(
                input, (self.receptive_field-in_len, 0, 0, 0))
        else:
            x = input
        x = self.start_conv(x)

        skip = 0
        if self.gcn_bool and self.addaptadj and self.supports is not None:


            gwadp = F.softmax(
                F.relu(torch.mm(self.nodevec1, self.nodevec2)), dim=1)

            new_supports = self.supports + [gwadp] # pretrained graph in DCRNN and self-adaptive graph in GWNet

            # Construction of dynamic frequency domain graph
            xn1 = input[:, 0, :, -self.seq_len:]

            T_D = self.T_i_D_emb[(data[:, :, :, 1] * 288).type(torch.LongTensor)][:, -1, :, :]
            D_W = self.D_i_W_emb[(data[:, :, :, 2] * 7).type(torch.LongTensor)][:, -1, :, :]
            # G_E = self.G_emb[(data[:, :, :, 3]).type(torch.LongTensor)][:, -1, :, :]
            # node2vec_emb = data[:, -1, :, 3:]
            # attended_emb = self.self_attention(node2vec_emb, node2vec_emb, node2vec_emb)
            pe_emb = data[:, -1, :, -64:]  # Форма: (B, N, 64)
            pe_emb = torch.matmul(pe_emb, self.pe)
            # attended_emb = self.self_attention(pe_emb, pe_emb, pe_emb)
            # pe_expanded = self.pe.unsqueeze(0).repeat(xn1.size(0), 1, 1)  # Расширяем PE до (B, N, 64)

            xn1 = torch.fft.rfft(xn1, dim=-1)
            xn1 = torch.abs(xn1)

            xn1 = torch.nn.functional.normalize(xn1, p=2.0, dim=1, eps=1e-12, out=None)
            xn1 = torch.nn.functional.normalize(xn1, p=2.0, dim=2, eps=1e-12, out=None) * self.a

            xn1 = torch.matmul(xn1, self.Ex1)

            xn1k = cat(xn1, self.node1)
            x_n1 = torch.cat([xn1k, T_D, D_W, pe_emb], dim=2)
            x1 = torch.bmm(x_n1.permute(1,0,2),self.Wd).permute(1,0,2)
            x1 = torch.relu(x1)
            x1k = self.layersnorm(x1)
            x1k = self.drop(x1k)
            adp = self.convt(x1k, self.Wxabs)
            adj = torch.bmm(adp, x1.permute(0, 2, 1))
            adp = torch.relu(adj)
            adp = dy_mask_graph(adp, self.subgraph_size)
            adp = F.softmax(adp, dim=2)
            new_supports = new_supports + [adp]

        # WaveNet layers
        for i in range(self.blocks * self.layers):

            # dilated convolution
            residual = x
            filter = self.filter_convs[i](residual)
            filter = torch.tanh(filter)
            gate = self.gate_convs[i](residual)
            gate = torch.sigmoid(gate)
            x = filter * gate

            # parametrized skip connection
            s = x
            s = self.skip_convs[i](s)
            try:
                skip = skip[:, :, :,  -s.size(3):]
            except:
                skip = 0
            skip = s + skip

            if self.gcn_bool and self.supports is not None:
                if self.addaptadj:
                    x = self.gconv[i](x, new_supports)

                else:
                    x = self.gconv[i](x, self.supports)
            else:
                x = self.residual_convs[i](x)
            x = x + residual[:, :, :, -x.size(3):]

            x = self.bn[i](x)

        x = F.relu(skip)
        x = F.relu(self.end_conv_1(x))
        x = self.end_conv_2(x)
        return x

### Инициализация модели

In [9]:
# Определение устройства
device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'
print(device)

# # Обновление модели, данных и вычислений
# supports = [torch.tensor(adj, dtype=torch.float32)]
# model = DFDGCN(num_nodes=N, supports=supports, in_dim=C, out_dim=pred_len, add_channels=64).to(device)
# criterion = nn.MSELoss()  # Функция потерь
# optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
# print(model)

# Обновление модели, данных и вычислений
adj = torch.tensor(adj, dtype=torch.float32).to(device)
model = QGNNTrafficPredictor(adj, 
                 num_nodes=N, 
                 input_dim=1, 
                 hidden_dim=64, 
                 output_dim=1, 
                 num_layers=2, 
                 pre_len=pred_len, 
                 space='r', 
                 dropout=0.5, 
                 directed=False, 
                 add_c=1+10,
                 emb_dim=325).to(device)
criterion = nn.MSELoss()  # Функция потерь
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
print(model)

cuda
QGNNTrafficPredictor(
  (embedding): Embedding(325, 5)
  (T_i_D_emb): Embedding(288, 5)
  (D_i_W_emb): Embedding(7, 5)
  (qgnn_layers): ModuleList(
    (0-1): 2 x QGNNLayer(
      (dropout): Dropout(p=0.5, inplace=False)
      (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (temporal_layer): GRU(75, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


### Инициализация TensorBoard и функция запуска обучения

In [11]:
def compute_metrics(output, target):
    """Вычисление метрик MAE, RMSE, MAPE."""
    abs_error = torch.abs(output - target).sum().item()
    mae = abs_error / len(target)
    rmse = ((output - target) ** 2).sum().item() / len(target)
    rmse = rmse ** 0.5
    mape = (abs_error / torch.abs(target).sum().item()) if torch.abs(target).sum().item() != 0 else 0
    return mae, rmse, mape

def train_val_test_model(model, train_loader, val_loader, test_loader, epochs, writer):
    best_val_loss = float('inf')

    for epoch in range(epochs):
        # === Тренировка ===
        model.train()
        train_loss, train_mae, train_rmse, train_mape = 0.0, 0.0, 0.0, 0.0
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} - Training", leave=False)

        for x, y in train_loader_tqdm:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(x).squeeze(-1)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * x.size(0)

            mae, rmse, mape = compute_metrics(output, y)
            train_mae += mae
            train_rmse += rmse
            train_mape += mape

        train_loss /= len(train_loader.dataset)
        train_mae /= len(train_loader)
        train_rmse /= len(train_loader)
        train_mape /= len(train_loader)

        writer.add_scalar("Loss/Train", train_loss, epoch + 1)
        writer.add_scalar("MAE/Train", train_mae, epoch + 1)
        writer.add_scalar("RMSE/Train", train_rmse, epoch + 1)
        writer.add_scalar("MAPE/Train", train_mape, epoch + 1)

        # === Валидация ===
        model.eval()
        val_loss, val_mae, val_rmse, val_mape = 0.0, 0.0, 0.0, 0.0
        val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{epochs} - Validation", leave=False)

        with torch.no_grad():
            for x, y in val_loader_tqdm:
                x, y = x.to(device), y.to(device)
                output = model(x).squeeze(-1)
                loss = criterion(output, y)
                val_loss += loss.item() * x.size(0)

                mae, rmse, mape = compute_metrics(output, y)
                val_mae += mae
                val_rmse += rmse
                val_mape += mape

        val_loss /= len(val_loader.dataset)
        val_mae /= len(val_loader)
        val_rmse /= len(val_loader)
        val_mape /= len(val_loader)

        writer.add_scalar("Loss/Validation", val_loss, epoch + 1)
        writer.add_scalar("MAE/Validation", val_mae, epoch + 1)
        writer.add_scalar("RMSE/Validation", val_rmse, epoch + 1)
        writer.add_scalar("MAPE/Validation", val_mape, epoch + 1)

        # === Тестирование ===
        test_loss, test_mae, test_rmse, test_mape = 0.0, 0.0, 0.0, 0.0
        test_loader_tqdm = tqdm(test_loader, desc=f"Epoch {epoch + 1}/{epochs} - Testing", leave=False)

        with torch.no_grad():
            for x, y in test_loader_tqdm:
                x, y = x.to(device), y.to(device)
                output = model(x).squeeze(-1)
                loss = criterion(output, y)
                test_loss += loss.item() * x.size(0)

                mae, rmse, mape = compute_metrics(output, y)
                test_mae += mae
                test_rmse += rmse
                test_mape += mape

        test_loss /= len(test_loader.dataset)
        test_mae /= len(test_loader)
        test_rmse /= len(test_loader)
        test_mape /= len(test_loader)

        writer.add_scalar("Loss/Test", test_loss, epoch + 1)
        writer.add_scalar("MAE/Test", test_mae, epoch + 1)
        writer.add_scalar("RMSE/Test", test_rmse, epoch + 1)
        writer.add_scalar("MAPE/Test", test_mape, epoch + 1)

        # Сохранение лучшей модели
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"{writer.log_dir}best_model.pth")

    writer.close()

### Обучение, валидация и тестирование

In [12]:
# Инициализация TensorBoard
writer = SummaryWriter(log_dir=f"runs/T-GCN/speed_GCN_closeness(one_hot)/")
train_val_test_model(model, train_loader, val_loader, test_loader, epochs=50, writer=writer)